In [22]:
# Import Dependencies
import pandas as pd
import numpy as np

# for geo charting
import folium

In [23]:
# indexfreedom file
freedom_csv_path = "data/indexfreedom.csv"

In [24]:
# READ freedom.csv
freedom_df = pd.read_csv(freedom_csv_path, encoding="utf-8")
freedom_df.head()

,year,ISO_code,countries,region,pf_rol_procedural,pf_rol_civil,pf_rol_criminal,pf_rol,pf_ss_homicide,pf_ss_disappearances_disap,...,ef_regulation_business_bribes,ef_regulation_business_licensing,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,ef_score,ef_rank,hf_score,hf_rank,hf_quartile
0,2016,ALB,Albania,Eastern Europe,6.661503,4.547244,4.666508,5.291752,8.920429,10.0,...,4.050196,7.324582,7.074366,6.705863,6.906901,7.54,34.0,7.568140,48.0,2.0
1,2016,DZA,Algeria,Middle East & North Africa,NaN,NaN,NaN,3.819566,9.456254,10.0,...,3.765515,8.523503,7.029528,5.676956,5.268992,4.99,159.0,5.135886,155.0,4.0
2,2016,AGO,Angola,Sub-Saharan Africa,NaN,NaN,NaN,3.451814,8.060260,5.0,...,1.945540,8.096776,6.782923,4.930271,5.518500,5.17,155.0,5.640662,142.0,4.0
3,2016,ARG,Argentina,Latin America & the Caribbean,7.098483,5.791960,4.343930,5.744791,7.622974,10.0,...,3.260044,5.253411,6.508295,5.535831,5.369019,4.84,160.0,6.469848,107.0,3.0
4,2016,ARM,Armenia,Caucasus & Central Asia,NaN,NaN,NaN,5.003205,8.808750,10.0,...,4.575152,9.319612,6.491481,6.797530,7.378069,7.57,29.0,7.241402,57.0,2.0


In [25]:
''# copy the freedom_df and make 'coutnries the index
freedom_country_df = freedom_df.copy()
#freedom_country_df.set_index(['ISO_code'], inplace=True)
freedom_country_df.head()

,year,ISO_code,countries,region,pf_rol_procedural,pf_rol_civil,pf_rol_criminal,pf_rol,pf_ss_homicide,pf_ss_disappearances_disap,...,ef_regulation_business_bribes,ef_regulation_business_licensing,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,ef_score,ef_rank,hf_score,hf_rank,hf_quartile
0,2016,ALB,Albania,Eastern Europe,6.661503,4.547244,4.666508,5.291752,8.920429,10.0,...,4.050196,7.324582,7.074366,6.705863,6.906901,7.54,34.0,7.568140,48.0,2.0
1,2016,DZA,Algeria,Middle East & North Africa,NaN,NaN,NaN,3.819566,9.456254,10.0,...,3.765515,8.523503,7.029528,5.676956,5.268992,4.99,159.0,5.135886,155.0,4.0
2,2016,AGO,Angola,Sub-Saharan Africa,NaN,NaN,NaN,3.451814,8.060260,5.0,...,1.945540,8.096776,6.782923,4.930271,5.518500,5.17,155.0,5.640662,142.0,4.0
3,2016,ARG,Argentina,Latin America & the Caribbean,7.098483,5.791960,4.343930,5.744791,7.622974,10.0,...,3.260044,5.253411,6.508295,5.535831,5.369019,4.84,160.0,6.469848,107.0,3.0
4,2016,ARM,Armenia,Caucasus & Central Asia,NaN,NaN,NaN,5.003205,8.808750,10.0,...,4.575152,9.319612,6.491481,6.797530,7.378069,7.57,29.0,7.241402,57.0,2.0


In [26]:
freedom_country_df['year'].value_counts()

2016    162
2015    162
2014    162
2013    162
2012    162
2011    162
2010    162
2009    162
2008    162
Name: year, dtype: int64

# Calcaulate % Change in hf_score by country between 2008 and 2016

## Set up Pivot Table
* ISO_code is Index on rows because the Folium package wants it instead of country name

In [27]:
# groupby iso_code and year 
freeISO_codeYear_gb = freedom_country_df.groupby(['ISO_code', 'year'])

## Filter Groupby by a relevant column (e.g hf_score)
temp_ds = freeISO_codeYear_gb['hf_score'].mean()

## take means to setup data frame
## even though means off of one year of data for each ISO_code is redundant
hfscore_mean_df = temp_ds.reset_index()

#print(f"{ISO_code_hfscore_mean_df.head()}")

In [28]:
# Pivot the data by country x year
hfscore_pivot_df = pd.pivot_table(hfscore_mean_df, index="ISO_code", columns="year")
hfscore_pivot_df

# Data column munging to clean df up and easier to use
# # https://stackoverflow.com/questions/22233488/pandas-drop-a-level-from-a-multi-level-column-index
# # https://stackoverflow.com/questions/36752439/how-do-i-flatten-a-python-pandas-pivot-table-and-manipulate-the-column-names
## drop the hf_score level and lable 
hfscore_pivot_df.columns = hfscore_pivot_df.columns.droplevel(0)

## drop the 'year' name of the column axis
hfscore_pivot_df.columns.name = None

## flatten the columns
hfscore_pivot_df.reset_index(inplace=True)

## set the index back to country (may not do this step)
hfscore_pivot_df.set_index('ISO_code', drop=True, inplace=True)

hfscore_pivot_df.head()

,2008,2009,2010,2011,2012,2013,2014,2015,2016
ISO_code,,,,,,,,,
AGO,4.823528,5.206898,5.237695,5.354951,5.832180,5.994839,5.618964,5.786479,5.640662
ALB,7.498849,7.585134,7.553625,7.502044,7.435191,7.389597,7.615083,7.558539,7.568140
ARE,6.784866,6.481551,6.710885,6.264678,6.287808,6.458097,6.419286,6.275228,6.286675
ARG,7.035120,6.997731,6.885658,6.855326,6.569287,6.570844,6.207145,6.502548,6.469848
ARM,7.455280,7.479161,7.488132,7.560340,7.559298,7.396940,7.307986,7.230887,7.241402


## Calculate Percent Change by country

In [29]:
# Calculate percent change in hf_score and add to pivot
hfscore_pivot_df['%-change hf_score'] = \
                (100 * (hfscore_pivot_df[2016] - hfscore_pivot_df[2008]) 
                        / hfscore_pivot_df[2008])
        
#hfscore_pivot_df

# Visualising Geospatial data with Python
Using Folium to visualize geospation data with Python is taken from this source: 
* https://medium.com/datadriveninvestor/visualising-geospatial-data-with-python-d3b1c519f31
* http://localhost:8888/tree/Visualising-Geospatial-data-with-Python-master 

The above interface is deprecated but still works
upgrade to use class 
* https://github.com/python-visualization/folium/blob/master/examples/GeoJSON_and_choropleth.ipynb
* https://en.wikipedia.org/wiki/GeoJSON



## Setting data for plotting.
Create a data frame with just the country codes and the values we want plotted.

In [33]:
# #hist_indicator =  'Life expectancy at birth'
# hist_year = 2016
# #mask1 = data['IndicatorName'].str.contains(hist_indicator) 
# mask2 = freedom_country_df['year'].isin([hist_year])

# apply our mask
#stage = freeRegionYear_mean_df[mask1 & mask2]
stage = hfscore_pivot_df.reset_index()
stage.head()

,ISO_code,2008,2009,2010,2011,2012,2013,2014,2015,2016,%-change hf_score
0,AGO,4.823528,5.206898,5.237695,5.354951,5.832180,5.994839,5.618964,5.786479,5.640662,16.940596
1,ALB,7.498849,7.585134,7.553625,7.502044,7.435191,7.389597,7.615083,7.558539,7.568140,0.924027
2,ARE,6.784866,6.481551,6.710885,6.264678,6.287808,6.458097,6.419286,6.275228,6.286675,-7.342682
3,ARG,7.035120,6.997731,6.885658,6.855326,6.569287,6.570844,6.207145,6.502548,6.469848,-8.035004
4,ARM,7.455280,7.479161,7.488132,7.560340,7.559298,7.396940,7.307986,7.230887,7.241402,-2.868821


In [34]:
data_to_plot = stage[['ISO_code','%-change hf_score']]
data_to_plot.head()

,ISO_code,%-change hf_score
0,AGO,16.940596
1,ALB,0.924027
2,ARE,-7.342682
3,ARG,-8.035004
4,ARM,-2.868821


## Label for the map

In [45]:
legendTitle = "%-Change in HF Score by Country"

# Visualising the Map

In [46]:
## Country coordinates for plotting
country_geo = 'data/world-countries.json'

In [47]:
map = folium.Map(location=[100, 0], zoom_start=1.5)

In [48]:
# choropleth maps bind Pandas Data Frames and json geometries.
map.choropleth(geo_data=country_geo, data=data_to_plot,
             columns=['ISO_code','%-change hf_score'],
             key_on='feature.id',
             fill_color='RdYlBu', fill_opacity=0.7, line_opacity=0.2,
             legend_name=legendTitle)

In [49]:
map.save('freedomplot_percChange_hfscore.html')

In [50]:
# Import the Folium interactive html file
from IPython.display import HTML
HTML('<iframe src=freedomplot_percChange_hfscore.html width=700 height=450></iframe>')